In [6]:
from numpy.linalg import solve
import numpy as np

In [7]:
# This code was inspiried by the blog of Ethan Rosenthal

In [8]:
# ratings is ndarray: user_items asscociated matrix
ratings = None

# users_num, items_num = ratings.shape

# the number of latent factors in the matrix factorization model
factors_num = 10

# regularization parameter for item factors
item_reg = 0.0

# regularization parameter for user factors
user_reg = 0.0

printing = False

In [9]:
# This function solves the latent_vectors of the matrix factorization
# The objective function here has already been taken derivatives
def als(latent_vectors, fixed_vecs, ratings, lbd, output = 'user'):
    """
    latent_vectors: the function returns latent_vectors, a tall and thin vector
    fixed_vecs: at each iteration, fix the fixed_vecs, and solve latent_vector
    lbd: regularization term lambda in order to prevent overfitting
    """
    if output == 'user': # trying to fix item vectors and solve user vectors
        QTQ = fixed_vecs.T.dot(fixed_vecs) # items by items matrix
        lambdaI = np.eye(QTQ.shape[0]) * lbd 
        
        for u in range(latent_vectors.shape[0]):
            latent_vectors[u, :] = solve((QTQ + lambdaI), ratings[u, :].dot(fixed_vecs)) # AX = b
    
    elif output == 'item': # trying to fix user vectors and solve item vectors
        PTP = fixed_vecs.T.dot(fixed_vecs) # user by user matrix
        lambdaI = np.eye(PTP.shape[0]) * lbd
        
        for i in range(latent_vectors.shape[0]):
            latent_vectors[i, :] = solve((PTP + lambdaI), ratings[:, i].T.dot(fixed_vecs))# AX = b
    return latent_vectors                                         

In [10]:
def train(iter_num = 20):
    # initialize two random vectors 
    user_vecs = np.random.random((users_num, factors_num))
    item_vecs = np.random.random((items_num, factors_num))
    
    partial_train(iter_num)

In [13]:
# normally there're two stopping criteria: the number of iterations, and the discrepancy between the result and actual target
# here we use the number of iterations
# a common choice of the number of iterations is below 40
def partial_train(iter_num):
    i = 1
    while i <= iter_num:
        if i % 10 == 0 and printing:
            print('current iteration', i)
        user_vecs = als(user_vecs, item_vecs, ratings, user_reg, output='user')
        item_vecs = als(item_vecs, user_vecs, ratings, item_reg, output='item')
        i += 1

In [14]:
def predict():
    predictions = np.zeros((user_vecs.shape[0], item_vecs.shape[0]))
    for u in range(user_vecs.shape[0]):
        for i in range(item_vecs.shape[0]):
            predictions[u, i] = predict(u, i)
    return predictions